In [1]:
import psycopg2
import pandas as pd
import json
import folium
import branca.colormap as cm

In [ ]:
# Load the GeoJSON file with county boundaries
geojson_file_path = 'Boundaries.geojson'  

# Load the GeoJSON data
with open(geojson_file_path) as f:
    geojson_data = json.load(f)

In [ ]:
# Function to get coordinates for a county
def get_coordinates_for_county(county_name):
    # Iterate through GeoJSON features to find the matching county
    for feature in geojson_data['features']:
        if feature['properties']['name'] == county_name:  
            # Extract the coordinates of the boundary
            return feature['geometry']['coordinates']
    # Default to Ohio center if county is not found
    return [40.4173, -82.9071]  # Default Ohio center coordinates

# Function to connect to PostgreSQL and retrieve data
def get_data_from_postgres(query):
    try:
        # Use the credentials from config.py
        connection = psycopg2.connect(
            host=db_config['host'],
            database=db_config['database'],
            user=db_config['user'],
            password=db_config['password'],
            port=db_config['port']
        )
        cursor = connection.cursor()
        cursor.execute(query)
        data = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(data, columns=columns)
        return df

    except Exception as e:
        print(f"Error: {e}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

In [ ]:
# Query to fetch data for the years 2020, 2021, 2022, and 2023
query = """
    SELECT county_name, year, deaths 
    FROM overdose_deaths 
    WHERE year IN (2020, 2021, 2022, 2023)
"""
overdose_data = get_data_from_postgres(query)

# Iterate over the rows of the DataFrame
for index, row in overdose_data.iterrows():
    # Extract the relevant information
    county_name = row['county_name']
    deaths = row['deaths']
    year = row['year']  # Get the year from the dataset

    # Fetch the coordinates for the county
    county_coordinates = get_coordinates_for_county(county_name)

    # Print the result with the county, year, deaths, and coordinates
    print(f"County: {county_name}, Year: {year}, Deaths: {deaths}, Coordinates: {county_coordinates}")


In [ ]:
# Split the data by year
data_2020 = overdose_data[overdose_data["year"] == 2020]
data_2021 = overdose_data[overdose_data["year"] == 2021]
data_2022 = overdose_data[overdose_data["year"] == 2022]
data_2023 = overdose_data[overdose_data["year"] == 2023]


In [ ]:
# Initialize map over Ohio
map =  folium.Map(location=[39.9612, -82.9988], zoom_start=8)

# Creates colormap scale and legend
colormap = cm.linear.YlOrRd_05.scale(0,10000).to_step(10)
colormap.caption = "Provisional Drug Overdose Deaths"
map.add_child(colormap)

# Function to create each overlay layer
# name = name of layer
# data = dataframe with county name, death count, geometry of boundaries
# startingOverlay = Boolean for if layer is shown at map initialization
def yearOverlay(name, data, startingOverlay):
    
    # Creates series of death counts with county name as index. Used for colormap.
    series = data.set_index("COUNTYNAME")["Provisional Drug Overdose Deaths"]

    # Creates overlay layer
    folium.GeoJson(data,
               name = name,
               show = startingOverlay,
               style_function=lambda feature: {
                   "fillColor": colormap(series[feature["properties"]["COUNTYNAME"]]), # Uses colormap scaling to select fill color
                   "fillOpacity": .6,
                   "color": "black",
                   "weight": 2
               },
               popup=folium.GeoJsonPopup(fields=["COUNTYNAME", "Provisional Drug Overdose Deaths"], aliases=["County", "Deaths"]),
               popup_keep_highlighted=True,
               tooltip=folium.GeoJsonTooltip(fields=["COUNTYNAME"], aliases=["County"])
               ).add_to(map)

# Run function for each year of data
yearOverlay("2020", data_2020, True)
yearOverlay("2021", data_2021, False)
yearOverlay("2022", data_2022, False)
yearOverlay("2023", data_2023, False)

# Add layer control
folium.LayerControl().add_to(map)

# WHEN COMPLETE ADD
# map.save('ohio_overdose_map.html')
map